In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import h5py as h5
import scipy.io as sio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from tensorflow.keras.preprocessing import sequence
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten, Embedding, Dropout, Activation, Reshape
#from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, GlobalAveragePooling1D, TimeDistributed
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
#Over-sampling: SMOTE
#SMOTE (Synthetic Minority Oversampling TEchnique) consists of synthesizing elements for the minority class, 
#based on those that already exist. It works randomly picking a point from the minority class and computing 
#the k-nearest neighbors for this point.The synthetic points are added between the chosen point and its neighbors.
#We'll use ratio='minority' to resample the minority class.
# smote = SMOTE('minority')

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
#dataset = pd.read_csv('sample_data/data_CKD_MI.csv')

In [ ]:
dataset = pd.read_csv('sample_data/data_CKD_bigdata_refined.csv')


In [ ]:
dataset.head()

,sex,age,sido,height,weight,waist,vision_left,vision_right,listen_left,listen_right,BP_HIGH,BP_LWST,BLDS,TOT_CHOLE,TRIGLYCERIDE,HDL_CHOLE,LDL_CHOLE,HMG,OLIG_PROTE_CD,CREATININE,SGOT_AST,SGPT_ALT,GAMMA_GTP,SMK_STAT_TYPE_RSPS_CD,DRINK_OR_NOT,MOUTH_CHECK,BMI,GFR,stage
0,1.0,8.0,43.0,170.0,75.0,90.0,1.0,1.0,1.0,1.0,120.0,80.0,99.0,193.0,92.0,48.0,126.0,17.1,1.0,1.0,21.0,35.0,40.0,1.0,1.0,1.0,25.951557,86.886038,2
1,1.0,7.0,11.0,180.0,80.0,89.0,0.9,1.2,1.0,1.0,130.0,82.0,106.0,228.0,121.0,55.0,148.0,15.8,1.0,0.9,20.0,36.0,27.0,3.0,0.0,1.0,24.691358,100.644197,1
2,1.0,9.0,41.0,165.0,75.0,91.0,1.2,1.5,1.0,1.0,120.0,70.0,98.0,136.0,104.0,41.0,74.0,15.8,1.0,0.9,47.0,32.0,68.0,1.0,0.0,0.0,27.548209,95.928640,1
3,1.0,11.0,48.0,175.0,80.0,91.0,1.5,1.2,1.0,1.0,145.0,87.0,95.0,201.0,106.0,76.0,104.0,17.6,1.0,1.1,29.0,34.0,18.0,1.0,0.0,1.0,26.122449,73.203730,2
4,1.0,11.0,30.0,165.0,60.0,80.0,1.0,1.2,1.0,1.0,138.0,82.0,101.0,199.0,104.0,61.0,117.0,13.8,1.0,0.8,19.0,12.0,25.0,1.0,0.0,1.0,22.038567,105.714492,1


In [ ]:
dataset.shape

(1000000, 29)

In [ ]:
from sklearn.model_selection import train_test_split  

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_over,y_over, test_size=0.2, random_state = 2)

In [ ]:
#dataset.isnull().sum()

In [ ]:
#dataset.iloc[:,-1]
classes = {1:0, 2:1, 3:2, 4:3, 5:4}
dataset['stage']=dataset['stage'].replace(to_replace=classes)

In [ ]:
# Data preprocessing
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

In [ ]:
from imblearn.over_sampling import RandomOverSampler
#from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
oversample = RandomOverSampler(random_state=42)
#undersample = RandomUnderSampler(sampling_strategy='majority')
X_over, y_over = oversample.fit_resample(X, y)
#X_over,y_over = undersample.fit_resample(X, y)
#X_over,y_over = undersample.fit_resample(X_over,y_over)
#X_over,y_over = undersample.fit_resample(X_over,y_over)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
Counter(y)

Counter({0: 475386, 1: 486184, 2: 36377, 3: 1026, 4: 1027})

In [ ]:
Counter(y_over)

Counter({0: 486184, 1: 486184, 2: 486184, 3: 486184, 4: 486184})

In [ ]:
from sklearn.model_selection import train_test_split    
from sklearn.preprocessing import MinMaxScaler
X_train, X_test, y_train, y_test = train_test_split(X_over,y_over, test_size=0.2, random_state = 2)


In [ ]:
from tensorflow import keras

In [ ]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dense, SimpleRNN
model=Sequential()
# model.add(keras.layers.Dense (128,input_shape=(X_train.shape[1],),activation='relu'))
model.add(SimpleRNN(units=16, input_shape=(X_train.shape[1], 1), activation="relu"))
model.add(keras.layers.Dense (128,activation='relu')) #input layer
model.add(Dropout(0.2))
model.add(keras.layers.Dense (75,activation='relu')) #input layer
model.add(Dropout(0.2))
model.add(keras.layers.Dense (64,activation='relu')) #input layer
model.add(Dropout(0.2))
model.add(keras.layers.Dense (32,activation='relu')) #hidden
model.add(Dropout(0.2))
model.add(keras.layers.Dense (16,activation='relu')) #hidden
model.add(Dropout(0.2))
model.add(keras.layers.Dense (5,activation='softmax')) #output

In [ ]:
# model = Sequential()
# model.add(keras.layers.Dense (15,input_shape=(X_train.shape[1],),activation='relu')) #input layer
# model.add(keras.layers.Dense (10,activation='relu')) #hidden
# model.add(keras.layers.Dense (5,activation='softmax')) #outputa

In [ ]:
model.compile(
    optimizer = 'rmsprop',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)
model.fit(X_train,y_train,epochs=10,validation_data=(X_test, y_test))

Epoch 1/10


ValueError: ignored

In [ ]:
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
y_prob = model.predict(X_test)
y_classes = y_prob.argmax(axis=-1)

[[7.8335680e-23 1.3650260e-02 9.8381698e-01 2.5327762e-03 2.4759675e-08]
 [1.1209121e-25 2.7724716e-03 9.9613994e-01 1.0875859e-03 1.4879987e-09]
 [9.7429979e-01 1.7624462e-02 9.4868199e-08 1.5605401e-06 8.0741094e-03]
 ...
 [9.7429979e-01 1.7624462e-02 9.4868199e-08 1.5605401e-06 8.0741094e-03]
 [2.1120111e-28 5.9797632e-04 9.9892199e-01 4.8007251e-04 1.0011715e-10]
 [5.1998555e-26 2.2982184e-03 9.9671763e-01 9.8418188e-04 1.0694823e-09]]
[2 2 0 ... 0 2 2]


In [ ]:
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
mat = confusion_matrix(y_test,y_classes)
mat

array([[80307, 16944,     4,     1,     0],
       [    0, 89500,  7773,     0,     0],
       [    0,    26, 96778,   549,     0],
       [    0,     0,     0, 97274,     0],
       [ 1521,  2564,     0,  3043, 89900]])

In [ ]:
from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_test,y_classes, target_names=['Class 1', 'Class 2', 'Class 3','Class 4','Class 5']))


Classification Report

              precision    recall  f1-score   support

     Class 1       0.98      0.81      0.89     97256
     Class 2       0.81      0.94      0.87     97273
     Class 3       0.94      0.99      0.97     97353
     Class 4       0.90      1.00      0.95     97274
     Class 5       1.00      0.86      0.92     97028

    accuracy                           0.92    486184
   macro avg       0.93      0.92      0.92    486184
weighted avg       0.93      0.92      0.92    486184

